In [23]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os


In [24]:
prototxtPath=os.path.sep.join([r'E:\python\mask\face','deploy.prototxt'])
weightsPath=os.path.sep.join([r'E:\python\mask\face','res10_300x300_ssd_iter_140000.caffemodel'])


In [25]:
net=cv2.dnn.readNet(protopath,weightpath)

In [26]:
model=load_model(r'E:\python\mask\mask_detector.model')

In [27]:
image=cv2.imread(r'E:\python\mask\mask.JPG')

In [28]:
image

array([[[200, 177, 162],
        [201, 178, 163],
        [201, 178, 163],
        ...,
        [218, 200, 189],
        [218, 200, 189],
        [218, 200, 189]],

       [[201, 178, 163],
        [201, 178, 163],
        [202, 179, 164],
        ...,
        [218, 200, 189],
        [218, 200, 189],
        [218, 200, 189]],

       [[201, 178, 163],
        [201, 178, 163],
        [202, 179, 164],
        ...,
        [218, 200, 189],
        [218, 200, 189],
        [218, 200, 189]],

       ...,

       [[111, 111, 111],
        [254, 254, 254],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[111, 111, 111],
        [254, 254, 254],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[111, 111, 111],
        [254, 254, 254],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [29]:
image=img_to_array(image)

In [30]:
image.shape

(190, 265, 3)

In [31]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,213.0))

In [32]:
blob

array([[[[  96.        ,   96.825     ,   97.        , ...,
           114.        ,  114.        ,  114.        ],
         [  96.45      ,   96.90375   ,   97.31874   , ...,
           114.        ,  114.        ,  114.        ],
         [  97.        ,   97.        ,   97.70833   , ...,
           114.        ,  114.        ,  114.        ],
         ...,
         [   7.        ,  124.97499   ,  150.70833   , ...,
           151.        ,  151.        ,  151.        ],
         [   7.        ,  124.97499   ,  150.70833   , ...,
           151.        ,  151.        ,  151.        ],
         [   7.        ,  124.97499   ,  150.70833   , ...,
           151.        ,  151.        ,  151.        ]],

        [[   0.        ,    0.82499695,    1.        , ...,
            23.        ,   23.        ,   23.        ],
         [   0.44999695,    0.90374756,    1.3187408 , ...,
            23.        ,   23.        ,   23.        ],
         [   1.        ,    1.        ,    1.7083282 , .

In [33]:
net.setInput(blob)
detections=net.forward()

In [39]:
#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'w' is not defined